# Fine-Tuning BERT for Phishing URL Identification

In [1]:
!pip install transformers peft trl evaluate datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 544.8/544.8 kB 34.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 8.8 MB/s eta 0:00:00


In [2]:
from datasets import load_dataset
from peft import PeftConfig, LoraConfig, get_peft_model

from transformers import (
    AutoModelForSequenceClassification,
    AutoTokenizer,
    DataCollatorWithPadding,
    Trainer,
    TrainingArguments,
)

import evaluate
import numpy as np

In [3]:
dataset_dict = load_dataset("shawhin/phishing-site-classification")
dataset_dict

README.md: 0.00B [00:00, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/98.0k [00:00<?, ?B/s]

data/validation-00000-of-00001.parquet:   0%|          | 0.00/21.4k [00:00<?, ?B/s]

data/test-00000-of-00001.parquet:   0%|          | 0.00/24.5k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/2100 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/450 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/450 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'labels'],
        num_rows: 2100
    })
    validation: Dataset({
        features: ['text', 'labels'],
        num_rows: 450
    })
    test: Dataset({
        features: ['text', 'labels'],
        num_rows: 450
    })
})

In [5]:
model_path = "google-bert/bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_path)

id2label = {0: "benign", 1: "malicious"}
label2id = {"benign": 0, "malicious": 1}

model = AutoModelForSequenceClassification.from_pretrained(model_path,
                                            num_labels = 2,
                                            id2label = id2label,
                                            label2id = label2id)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [6]:
# set trainable params

# freeze all base model parameters
for name, param in model.base_model.named_parameters():
    param.requires_grad = False

# unfreeze base model pooling layers
for name, param in model.base_model.named_parameters():
    if "pooler" in name:
        param.requires_grad = True

In [7]:
# data preprocessing

def preprocess_text(examples):
  return tokenizer(examples['text'], truncation=True)

tokenized_dataset = dataset_dict.map(preprocess_text, batched=True)

# create data collator
data_collator = DataCollatorWithPadding(tokenizer=tokenizer) # to make sure very example in batch is of same size

Map:   0%|          | 0/2100 [00:00<?, ? examples/s]

Map:   0%|          | 0/450 [00:00<?, ? examples/s]

Map:   0%|          | 0/450 [00:00<?, ? examples/s]

In [12]:
# def eval metrics

accuracy = evaluate.load('accuracy')
auc_score = evaluate.load('roc_auc')

def compute_metrics(eval_pred):
    # get preds
    predictions, labels = eval_pred

    # apply softmax to get probabilities
    probabilities = np.exp(predictions) / np.exp(predictions).sum(-1,
                                                                  keepdims=True)

    # use probabilities of positive class for ROC AUC
    positive_class_probs = probabilities[:, 1]

    # compute auc
    auc = np.round(auc_score.compute(prediction_scores = positive_class_probs,
                                     references = labels)['roc_auc'], 3)

    # predict most probable class
    predicted_classes = np.argmax(probabilities, axis = 1)

    # compute accuracy
    acc = np.round(accuracy.compute(predictions = predicted_classes,
                                                references = labels)['accuracy'], 3)

    return {'accuracy': acc, 'AUC': auc}


In [13]:
# training params
lr = 2e-4
batch_size = 8
num_epochs = 10

training_args = TrainingArguments(
    output_dir = "bert-phishing-classifer_teacher",
    learning_rate = lr,
    per_device_train_batch_size = batch_size,
    per_device_eval_batch_size = batch_size,
    num_train_epochs = num_epochs,
    weight_decay = 0.01,
    logging_strategy = "epoch",
    eval_strategy = "epoch",
    save_strategy = "epoch",
    load_best_model_at_end = True,
)

In [14]:
trainer = Trainer (
    model = model,
    args = training_args,
    train_dataset = tokenized_dataset["train"],
    eval_dataset = tokenized_dataset["test"],
    tokenizer = tokenizer,
    data_collator = data_collator,
    compute_metrics = compute_metrics,
)

trainer.train()

/tmp/ipython-input-1165136351.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer (
/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: shivangshandilya83 (shivangshandilya83-independent) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss,Accuracy,Auc
1,0.503700,0.384133,0.811000,0.911000
2,0.407700,0.338099,0.847000,0.930000
3,0.357700,0.313102,0.853000,0.939000
4,0.355700,0.353034,0.849000,0.945000
5,0.352200,0.338283,0.862000,0.948000
6,0.348700,0.292409,0.873000,0.950000
7,0.333800,0.287450,0.876000,0.950000
8,0.314200,0.288594,0.869000,0.950000
9,0.313700,0.284354,0.873000,0.951000
10,0.313500,0.288827,0.871000,0.951000


TrainOutput(global_step=2630, training_loss=0.36009377889306826, metrics={'train_runtime': 214.0155, 'train_samples_per_second': 98.124, 'train_steps_per_second': 12.289, 'total_flos': 706603239165360.0, 'train_loss': 0.36009377889306826, 'epoch': 10.0})